# Semantic Caching for LLMs

RedisVL provides an ``SemanticCache`` interface to turn Redis into a semantic cache to store responses to previously asked questions. This reduces the number of requests and tokens sent to the Large Language Models (LLM) service, decreasing costs and enhancing application throughput (by reducing the time taken to generate responses).

This notebook will go over how to use Redis as a Semantic Cache for your applications

First, we will import [OpenAI](https://platform.openai.com) to use their API for responding to user prompts. We will also create a simple `ask_openai` helper method to assist.

In [1]:
import os
import openai
import getpass

os.environ["TOKENIZERS_PARALLELISM"] = "False"

api_key = os.getenv("OPENAI_API_KEY") or getpass.getpass("Enter your OpenAI API key: ")

openai.api_key = api_key

def ask_openai(question: str) -> str:
    response = openai.Completion.create(
      engine="gpt-3.5-turbo-instruct",
      prompt=question,
      max_tokens=200
    )
    return response.choices[0].text.strip()

In [2]:
# Test
print(ask_openai("What is the capital of France?"))

The capital of France is Paris.


## Initializing ``SemanticCache``

``SemanticCache`` will automatically create an index within Redis upon initialization for the semantic cache content.

In [3]:
from redisvl.llmcache import SemanticCache

llmcache = SemanticCache(
    name="llmcache",                     # underlying search index name
    prefix="llmcache",                   # redis key prefix for hash entries
    redis_url="redis://localhost:6379",  # redis connection url string
    distance_threshold=0.1               # semantic cache distance threshold
)

/Users/tyler.hutcherson/RedisVentures/redisvl/.venv/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
# look at the index specification created for the semantic cache lookup
!rvl index info -i llmcache



Index Information:
╭──────────────┬────────────────┬──────────────┬─────────────────┬────────────╮
│ Index Name   │ Storage Type   │ Prefixes     │ Index Options   │   Indexing │
├──────────────┼────────────────┼──────────────┼─────────────────┼────────────┤
│ llmcache     │ HASH           │ ['llmcache'] │ []              │          0 │
╰──────────────┴────────────────┴──────────────┴─────────────────┴────────────╯
Index Fields:
╭───────────────┬───────────────┬────────┬────────────────┬────────────────╮
│ Name          │ Attribute     │ Type   │ Field Option   │   Option Value │
├───────────────┼───────────────┼────────┼────────────────┼────────────────┤
│ prompt        │ prompt        │ TEXT   │ WEIGHT         │              1 │
│ response      │ response      │ TEXT   │ WEIGHT         │              1 │
│ prompt_vector │ prompt_vector │ VECTOR │                │                │
╰───────────────┴───────────────┴────────┴────────────────┴────────────────╯


## Basic Cache Usage

In [5]:
question = "What is the capital of France?"

In [6]:
# Check the semantic cache -- should be empty
if response := llmcache.check(prompt=question):
    print(response)
else:
    print("Empty cache")

Empty cache


Our initial cache check should be empty since we have not yet stored anything in the cache. Below, store the `question`,
proper `response`, and any arbitrary `metadata` (as a python dictionary object) in the cache.

In [7]:
# Cache the question, answer, and arbitrary metadata
llmcache.store(
    prompt=question,
    response="Paris",
    metadata={"city": "Paris", "country": "france"}
)

'llmcache:115049a298532be2f181edb03f766770c0db84c22aff39003fec340deaec7545'

In [8]:
# Check the cache again
if response := llmcache.check(prompt=question, return_fields=["prompt", "response", "metadata"]):
    print(response)
else:
    print("Empty cache")

[{'id': 'llmcache:115049a298532be2f181edb03f766770c0db84c22aff39003fec340deaec7545', 'vector_distance': '8.34465026855e-07', 'prompt': 'What is the capital of France?', 'response': 'Paris', 'metadata': {'city': 'Paris', 'country': 'france'}}]


In [9]:
# Check for a semantically similar result
question = "What actually is the capital of France?"
llmcache.check(prompt=question)[0]['response']

'Paris'

In [10]:
# Widen the semantic distance threshold
llmcache.set_threshold(0.3)

In [11]:
# Really try to trick it by asking around the point
# But is able to slip just under our new threshold
question = "What is the capital city of the country in Europe that also has a city named Nice?"
llmcache.check(prompt=question)[0]['response']

'Paris'

In [12]:
# Invalidate the cache completely by clearing it out
llmcache.clear()

# should be empty now
llmcache.check(prompt=question)

[]

## Simple performance testing

Next, we will measure the speedup obtained by using ``SemanticCache``. We will use the ``time`` module to measure the time taken to generate responses with and without ``SemanticCache``.

In [13]:
import time


def answer_question(question: str) -> str:
    """Helper function to answer a simple question using OpenAI with a wrapper
    check for the answer in the semantic cache first.

    Args:
        question (str): User input question.

    Returns:
        str: Response.
    """
    results = llmcache.check(prompt=question)
    if results:
        return results[0]["response"]
    else:
        answer = ask_openai(question)
        return answer

In [14]:
start = time.time()
# asking a question -- openai response time
answer = answer_question("What is the capital of France?")
end = time.time()

print(f"Without caching, a call to openAI to answer this simple question took {end-start} seconds.")

Without caching, a call to openAI to answer this simple question took 0.7188658714294434 seconds.


In [15]:
llmcache.store(prompt="What is the capital of France?", response="Paris")

'llmcache:115049a298532be2f181edb03f766770c0db84c22aff39003fec340deaec7545'

In [16]:
cached_start = time.time()
cached_answer = answer_question("What is the capital of France?")
cached_end = time.time()
print(f"Time Taken with cache enabled: {cached_end - cached_start}")
print(f"Percentage of time saved: {round(((end - start) - (cached_end - cached_start)) / (end - start) * 100, 2)}%")

Time Taken with cache enabled: 0.4411182403564453
Percentage of time saved: 38.64%


In [17]:
# check the stats of the index
!rvl stats -i llmcache


Statistics:
╭─────────────────────────────┬─────────────╮
│ Stat Key                    │ Value       │
├─────────────────────────────┼─────────────┤
│ num_docs                    │ 1           │
│ num_terms                   │ 7           │
│ max_doc_id                  │ 2           │
│ num_records                 │ 16          │
│ percent_indexed             │ 1           │
│ hash_indexing_failures      │ 0           │
│ number_of_uses              │ 9           │
│ bytes_per_record_avg        │ 5.25        │
│ doc_table_size_mb           │ 0.000134468 │
│ inverted_sz_mb              │ 8.01086e-05 │
│ key_table_size_mb           │ 2.76566e-05 │
│ offset_bits_per_record_avg  │ 8           │
│ offset_vectors_sz_mb        │ 1.33514e-05 │
│ offsets_per_term_avg        │ 0.875       │
│ records_per_doc_avg         │ 16          │
│ sortable_values_size_mb     │ 0           │
│ total_indexing_time         │ 0.996       │
│ total_inverted_index_blocks │ 25          │
│ vector_index_sz_mb 

In [18]:
# Clear the cache AND delete the underlying index
llmcache.delete()